In [1]:
from keras.applications.vgg16 import VGG16
from keras_vggface.vggface import VGGFace
import tensorflow as tf
import numpy as np

face_model = VGGFace(model='vgg16', 
                weights='vggface',
                input_shape=(224,224,3)) 
face_model.summary()

Using TensorFlow backend.


Model: "vggface_vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)     

In [2]:
for layer in face_model.layers:
    layer.trainable = False

In [3]:
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation

person_count = 2

last_layer = face_model.get_layer('pool5').output

x = Flatten(name='flatten')(last_layer)
x = Dense(1024, activation='relu', name='fc6')(x)
x = Dense(1024, activation='relu', name='fc7')(x)
out = Dense(person_count, activation='softmax', name='fc8')(x)

custom_face = Model(face_model.input, out)

In [10]:
from keras.preprocessing.image import ImageDataGenerator
batch_size = 5
train_path = 'cam_face'
eval_path = 'cam_eval_face'


In [11]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

valid_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
                        train_path,
                        target_size=(224,224),
                        batch_size=batch_size,
                        class_mode='sparse',
                        color_mode='rgb')

valid_generator = valid_datagen.flow_from_directory(
    directory=eval_path,
    target_size=(224, 224),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=True,
)

Found 19 images belonging to 2 classes.
Found 19 images belonging to 2 classes.


In [12]:
from keras.optimizers import SGD

custom_face.compile(loss='sparse_categorical_crossentropy',
                         optimizer=SGD(lr=1e-4, momentum=0.9),
                         metrics=['accuracy'])

history = custom_face.fit_generator(
        train_generator,
        validation_data=valid_generator,
        steps_per_epoch=49/batch_size,
        validation_steps=valid_generator.n,
        epochs=100)


Epoch 1/100
10/9 [==============================] - 24s 2s/step - loss: 0.6887 - accuracy: 0.5417 - val_loss: 0.6961 - val_accuracy: 0.5824
Epoch 2/100
10/9 [==============================] - 21s 2s/step - loss: 0.6836 - accuracy: 0.6383 - val_loss: 0.6777 - val_accuracy: 0.7000
Epoch 3/100
10/9 [==============================] - 20s 2s/step - loss: 0.6825 - accuracy: 0.7292 - val_loss: 0.6890 - val_accuracy: 0.7333
Epoch 4/100
10/9 [==============================] - 20s 2s/step - loss: 0.6804 - accuracy: 0.7872 - val_loss: 0.6626 - val_accuracy: 0.7889
Epoch 5/100
10/9 [==============================] - 18s 2s/step - loss: 0.6743 - accuracy: 0.9787 - val_loss: 0.6750 - val_accuracy: 0.8901
Epoch 6/100
10/9 [==============================] - 22s 2s/step - loss: 0.6669 - accuracy: 0.9375 - val_loss: 0.6654 - val_accuracy: 0.9667
Epoch 7/100
10/9 [==============================] - 19s 2s/step - loss: 0.6637 - accuracy: 0.9167 - val_loss: 0.6631 - val_accuracy: 1.0000
Epoch 8/100
10/9 [==

<strong> #note I could have over trained but sense there is not much data you need to train more</strong>

In [14]:
custom_face.evaluate_generator(generator=valid_generator)
        
custom_face.save('h5 or xml files/my_face.h5')

<h1><strong>Face cam</strong></h1>

In [15]:
from keras.models import load_model
from keras_vggface import utils
from keras_vggface.utils import preprocess_input
import cv2
import numpy as np

image_size = 224
device_id = 0 #camera_device id 

model = load_model('h5 or xml files/my_face.h5')


In [16]:
labels = dict(Trae=0,Janessa=1) 
print(labels)


{'Trae': 0, 'Janessa': 1}


In [26]:
cascade_classifier = cv2.CascadeClassifier('h5 or xml files/haarcascade_frontalface_default.xml')
camera = cv2.VideoCapture(device_id)

while camera.isOpened():
    ok, cam_frame = camera.read()
    if not ok:
        break
    
    gray_img=cv2.cvtColor(cam_frame, cv2.COLOR_BGR2GRAY)
    faces= cascade_classifier.detectMultiScale(gray_img, minNeighbors=5)

    for (x,y,w,h) in faces:
        cv2.rectangle(cam_frame,(x,y),(x+w,y+h),(255,255,0),2)
        roi_color = cam_frame [y:y+h, x:x+w]
        roi_color = cv2.cvtColor(roi_color, cv2.COLOR_BGR2RGB)
        roi_color = cv2.resize(roi_color, (image_size, image_size))
        image = roi_color.astype(np.float32, copy=False)
        image = np.expand_dims(image, axis=0)
        image = preprocess_input(image, version=1) # or version=2
        preds = model.predict(image)
        predicted_class=np.argmax(preds,axis=1)
         
          

        labels = (train_generator.class_indices)
        labels = dict((v,k) for k,v in labels.items())
        name = [labels[k] for k in predicted_class]
        
        cv2.putText(cam_frame,str(name), 
                    (x + 10, y + 10), cv2.FONT_HERSHEY_SIMPLEX, 1, 
                     (255,0,255), 2)
        
       
    cv2.imshow('video image', cam_frame)
    key = cv2.waitKey(30)
    if key == 27: # press 'ESC' to quit
        break
        
camera.release()
cv2.destroyAllWindows()

In [27]:
camera.release()
cv2.destroyAllWindows()